# Hypothesis Testing in Healthcare: Drug Safety

A pharmaceutical company GlobalXYZ has just completed a randomized controlled drug trial. To promote transparency and reproducibility of the drug's outcome, they (GlobalXYZ) have presented the dataset to your organization, a non-profit that focuses primarily on drug safety.

The dataset provided contained five adverse effects, demographic data, vital signs, etc. Your organization is primarily interested in the drug's adverse reactions. It wants to know if the adverse reactions, if any, are of significant proportions. It has asked you to explore and answer some questions from the data.

The dataset `drug_safety.csv` was obtained from [Hbiostat](https://hbiostat.org/data/) courtesy of the Vanderbilt University Department of Biostatistics. It contained five adverse effects: headache, abdominal pain, dyspepsia, upper respiratory infection, chronic obstructive airway disease (COAD), demographic data, vital signs, lab measures, etc. The ratio of drug observations to placebo observations is 2 to 1.

For this project, the dataset has been modified to reflect the presence and absence of adverse effects `adverse_effects` and the number of adverse effects in a single individual `num_effects`.

The columns in the modified dataset are: 

| Column | Description |
|--------|-------------|
|`sex` | The gender of the individual |
|`age` | The age of the individual |
|`week` | The week of the drug testing |
|`trx` | The treatment (Drug) and control (Placebo) groups | 
|`wbc` | The count of white blood cells |
|`rbc` | The count of red blood cells |
|`adverse_effects` | The presence of at least a single adverse effect |
|`num_effects` | The number of adverse effects experienced by a single individual |

The original dataset can be found [here](https://hbiostat.org/data/repo/safety.rda).

Your organization has asked you to explore and answer some questions from the data collected. See the project instructions.

In [100]:
import numpy as np
import pandas as pd
from statsmodels.stats.proportion import proportions_ztest
from scipy.stats import chi2_contingency
from scipy.stats import ttest_ind
from scipy.stats import mannwhitneyu
import pingouin
import seaborn as sns
import matplotlib.pyplot as plt

# load the dataset
drug_safety = pd.read_csv("drug_safety.csv")

## Determine if the proportion of adverse effects differ significantly between Drug and Placebo groups.


| | |
|-|-|
| Null Hypothesis (H₀) | The proportion of individuals with adverse effects is the same in both Drug and Placebo groups. p₁ = p₂ |
| Alternative Hypothesis (H₁) | The proportion of adverse effects differs between the two groups. p₁ ≠ p₂ |

In [84]:
drug_safety.head()

,age,sex,trx,week,wbc,rbc,adverse_effects,num_effects
0,62,male,Drug,0,7.3,5.1,No,0
1,62,male,Drug,1,NaN,NaN,No,0
2,62,male,Drug,12,5.6,5.0,No,0
3,62,male,Drug,16,NaN,NaN,No,0
4,62,male,Drug,2,6.6,5.1,No,0


In [85]:
drug_safety.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16103 entries, 0 to 16102
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   age              16103 non-null  int64  
 1   sex              16103 non-null  object 
 2   trx              16103 non-null  object 
 3   week             16103 non-null  int64  
 4   wbc              9128 non-null   float64
 5   rbc              9127 non-null   float64
 6   adverse_effects  16103 non-null  object 
 7   num_effects      16103 non-null  int64  
dtypes: float64(2), int64(3), object(3)
memory usage: 1006.6+ KB


We'll initially split the data into the two sets - drug and placebo, with a ratio of approx. 2:1 - we can assume the drug set will be around 9-11k total rows.

In [86]:
# seperate the data into drug and placebo participants
drug_split = drug_safety[drug_safety["trx"] == 'Drug']
drug_split.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10727 entries, 0 to 16094
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   age              10727 non-null  int64  
 1   sex              10727 non-null  object 
 2   trx              10727 non-null  object 
 3   week             10727 non-null  int64  
 4   wbc              6011 non-null   float64
 5   rbc              6011 non-null   float64
 6   adverse_effects  10727 non-null  object 
 7   num_effects      10727 non-null  int64  
dtypes: float64(2), int64(3), object(3)
memory usage: 754.2+ KB


In [87]:
placebo_split = drug_safety[drug_safety["trx"] == 'Placebo']
placebo_split.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5376 entries, 32 to 16102
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   age              5376 non-null   int64  
 1   sex              5376 non-null   object 
 2   trx              5376 non-null   object 
 3   week             5376 non-null   int64  
 4   wbc              3117 non-null   float64
 5   rbc              3116 non-null   float64
 6   adverse_effects  5376 non-null   object 
 7   num_effects      5376 non-null   int64  
dtypes: float64(2), int64(3), object(3)
memory usage: 378.0+ KB


In [88]:
# refine columns of each for purpose of test
drug = drug_split[["trx","adverse_effects"]]
placebo = placebo_split[["trx","adverse_effects"]]

drug.head(2)

,trx,adverse_effects
0,Drug,No
1,Drug,No


In [89]:
# check ratio of adverse effects to none in drug group
drug.groupby(['trx', 'adverse_effects']).size()

trx   adverse_effects
Drug  No                 9703
      Yes                1024
dtype: int64

In [90]:
# check ratio of adverse effects to none in placebo group
placebo.groupby(['trx', 'adverse_effects']).size()

trx      adverse_effects
Placebo  No                 4864
         Yes                 512
dtype: int64

In [91]:
# create variables for test
drug_yes = drug[(drug['trx'] == 'Drug') & (drug['adverse_effects'] == 'Yes')].shape[0]
drug_total = drug[drug['trx'] == 'Drug'].shape[0]
placebo_yes = placebo[(placebo['trx'] == 'Placebo') & (placebo['adverse_effects'] == 'Yes')].shape[0]
placebo_total = placebo[placebo['trx'] == 'Placebo'].shape[0]

yes_count = [drug_yes, placebo_yes]
total_count = [drug_total, placebo_total]

# run two proportion z-test
z_stat, two_sample_p_value = proportions_ztest(yes_count, total_count)

print(f"P-value: {two_sample_p_value:.4f}")

P-value: 0.9639


p-value ≈ 0.96 is way above any typical significance level. There’s no significant difference in the proportion of adverse effects between the Drug and Placebo groups — at least based on this dataset.

**We fail to reject the null hypothesis.**

In [92]:
alpha = 0.05
if two_sample_p_value < alpha:
    print("Reject the null hypothesis: There is a significant difference in the proportion of adverse effects between the Drug and Placebo groups.")
else:
    print("Fail to reject the null hypothesis: There’s no significant difference in the proportion of adverse effects between the Drug and Placebo groups.")

Fail to reject the null hypothesis: There’s no significant difference in the proportion of adverse effects between the Drug and Placebo groups.


---
## Find out if the number of adverse effects is independent of the treatment and control groups

In [99]:
# create a cross table for effect count vs participant groups
count_table = pd.crosstab(drug_safety['trx'], drug_safety['num_effects'])

# run the chi-squared test:
chi2_stat, num_effects_p_value, dof, expected = chi2_contingency(count_table)

print(f"P-value: {num_effects_p_value:.4f}")

P-value: 0.6150


In [95]:
alpha = 0.05
if num_effects_p_value < alpha:
    print("Reject the null hypothesis: number of effects is associated with treatment group.")
else:
    print("Fail to reject the null hypothesis: number of effects is independent of treatment group.")

Fail to reject the null hypothesis: number of effects is independent of treatment group.


---
## Find out if the number of adverse effects is independent of the treatment and control groups

In [96]:
# create a series of ages for each group
drug_age = drug_safety[drug_safety['trx'] == 'Drug']['age'].dropna()
placebo_age = drug_safety[drug_safety['trx'] == 'Placebo']['age'].dropna()

# run mann–whitney U test
stat, age_group_effects_p_value = mannwhitneyu(drug_age, placebo_age, alternative='two-sided')

print(f"P-value for age difference: {age_group_effects_p_value:.4f}")

P-value for age difference: 0.2570


In [97]:
alpha = 0.05
if age_group_effects_p_value < alpha:
    print("Reject H₀: Significant difference in age between groups.")
else:
    print("Fail to reject H₀: No significant difference in age between groups.")

Fail to reject H₀: No significant difference in age between groups.
